In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# webdriver_path = '/Users/xinyuepan/opt/anaconda3/envs/DS_Toolkit/bin:/Users/xinyuepan/opt/anaconda3/condabin:/usr/local/bin:/usr/local/sbin:/Library/Frameworks/Python.framework/Versions/3.8/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin'
driver = webdriver.Chrome(service=Service())

In [4]:
driver.get('https://filialen.migros.ch/de/filter:markets-%5Bsuper.voi.mp%5D/center:46.8182,8.2275/zoom:8/')
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))


<selenium.webdriver.remote.webelement.WebElement (session="f56a4276c3fc6b5229afdd2ff54b351a", element="DE3BE65B2DA64DB6DB6FFE971196449E_element_30")>

In [13]:
store_dict = {}

In [139]:
html_content = driver.page_source
soup = BeautifulSoup(html_content,"html.parser")

In [140]:
stores = soup.find_all('li',{'class':'sc-dPFVfm eHYuDp'})
for store in stores:
    store_name = store.find('span',{'class':'sc-eYVAha jlVxJL'}).text
    store_address = store.find('p',{'class':'sc-hlfjlA kiDzXg'}).text
    store_dict[store_name] = store_address
print(len(store_dict))
print(list(store_dict.keys())[-1])

771
teo Neuhegi


In [148]:
store_dict.items()

dict_items([('Aachtal', 'Schrofenstrasse 19, 8580 Amriswil'), ('Aadorf', 'Bahnhofstrasse 2, 8355 Aadorf'), ('Aarau - Igelweid', 'Igelweid 18, 5000 Aarau'), ('Aarau Bahnhof', 'Bahnhofplatz 3h, 5000 Aarau'), ('Abtwil - Säntispark', 'Wiesenbachstrasse 7, 9030 Abtwil SG'), ('Adligenswil', 'Obgardirain 1, 6043 Adligenswil'), ('Adliswil', 'Albisstrasse 10, 8134 Adliswil'), ('Aesch', 'Hauptstrasse 95, 4147 Aesch'), ('Affoltern a. A.', 'Oberdorfstrasse 4, 8910 Affoltern am Albis'), ('Agy', "route d'Agy 2, 1763 Granges-Paccot"), ('Aigle - Chablais Centre', 'Ch. Sous-le-Grand Pré 4, 1860 Aigle'), ('Aigle gare', 'Place de la gare 5, 1860 Aigle'), ('Allschwil - Paradies', 'Spitzwaldstrasse 215, 4123 Allschwil'), ('Allschwil - Ziegelei', 'Baslerstrasse 191, 4123 Allschwil'), ('Allschwil-Lindenplatz', 'Lindenplatz 1, 4123 Allschwil'), ('Altdorf', 'Gitschenstrasse 14, 6460 Altdorf'), ('Altstätten SG', 'Stossstrasse 2, 9450 Altstätten SG'), ('Amriswil', 'Rütistrasse 10a, 8580 Amriswil'), ('Andelfingen

In [151]:
migros_df = pd.DataFrame(store_dict.items(),columns=['name','address'])
migros_df

,name,address
0,Aachtal,"Schrofenstrasse 19, 8580 Amriswil"
1,Aadorf,"Bahnhofstrasse 2, 8355 Aadorf"
2,Aarau - Igelweid,"Igelweid 18, 5000 Aarau"
3,Aarau Bahnhof,"Bahnhofplatz 3h, 5000 Aarau"
4,Abtwil - Säntispark,"Wiesenbachstrasse 7, 9030 Abtwil SG"
...,...,...
766,Zürich - Witikon-Zentrum,"Witikonerstrasse 293, 8053 Zürich"
767,Zürich - Wollishofen,"Etzelstrasse 3, 8038 Zürich"
768,teo Bürglen,"Weinfelderstrasse 33a, 8575 Bürglen"
769,teo Kloten,"Lindenstrasse 18, 8302 Kloten"


In [157]:
def get_zip_code(address):
    address_list=address.split(', ')
    zip_code = address_list[-1].split()[0]
    return zip_code

In [158]:
migros_df['zip_code']=migros_df['address'].apply(get_zip_code)

In [159]:
migros_df.to_csv('test.csv')

In [185]:
zipcode_count_df = migros_df.groupby('zip_code').count().reset_index(names=['zip_code'])

zipcode_count_df.drop(columns=['address'],inplace=True)
zipcode_count_df.rename(columns={'name':'store_count'},inplace=True)
zipcode_count_df = zipcode_count_df.astype('int64')
type(zipcode_count_df)

pandas.core.frame.DataFrame

In [190]:
zurich_migros = zipcode_count_df[(zipcode_count_df['zip_code']>=8000) & (zipcode_count_df['zip_code']<8100)]
zurich_migros.to_csv('zurich_migros.csv')

In [189]:
zurich_migros

,zip_code,store_count
413,8001,3
414,8002,2
415,8003,1
416,8004,3
417,8005,4
418,8006,1
419,8008,4
420,8037,2
421,8038,2
422,8041,2
